In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

sys.path.append("helpers")
from helpers import LORIS_data_preparation

In [2]:
# LORIS saved query (all data)
full = pd.read_csv("../../diagnosis_predictor/data/raw/LORIS-release-10.csv", dtype=object)

In [3]:
df = LORIS_data_preparation.prepare_data(full)

/Users/home/Repos/hbn/HBN-scripts/exploration/helpers/LORIS_data_preparation.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full["ID"] = full_for_EID_check.iloc[:, 0]


In [4]:
age = "Basic_Demos,Age"
sex = "Basic_Demos,Sex"
spelling = "WIAT,WIAT_Spell_Stnd"

df[age] = np.floor(df[age].astype(float))

print(len(df[~(df[spelling].isnull())][spelling]))
print("6yo with WIAT: ", len(df[(df[age] == 6) & ~(df[spelling].isnull())][spelling]))
print("5yo with WIAT: ", len(df[(df[age] == 5) & ~(df[spelling].isnull())][spelling]))
print("5 and 6yo with WIAT: ", len(df[((df[age] == 5) | (df[age] == 6)) & ~(df[spelling].isnull())]))

all_wiat_cols = [x for x in df.columns if "WIAT" in x]
all_wisc_cols = [x for x in df.columns if "WISC" in x]
five_and_six_wiat = df[((df[age] == 5) | (df[age] == 6)) & ~(df[spelling].isnull())]
five_and_six_wiat[all_wiat_cols + all_wisc_cols + [age, sex]].to_csv("five_and_six_wiat.csv")

3682
6yo with WIAT:  386
5yo with WIAT:  249
5 and 6yo with WIAT:  635


In [5]:
print("Has Parent learning problems filled:", len(five_and_six_wiat[five_and_six_wiat["PreInt_FamHx,f_learn"].notna()]))
print("Has Spoke first word age filled:", len(five_and_six_wiat[five_and_six_wiat["PreInt_DevHx,skill_age_15"].notna() | five_and_six_wiat["PreInt_DevHx,skill_range_15"].notna()]))
print("Has Named objects age filled:", len(five_and_six_wiat[five_and_six_wiat["PreInt_DevHx,skill_age_16"].notna() | five_and_six_wiat["PreInt_DevHx,skill_range_16"].notna()]))
print("Has Put 2 or 3 words together age filled:", len(five_and_six_wiat[five_and_six_wiat["PreInt_DevHx,skill_age_17"].notna() | five_and_six_wiat["PreInt_DevHx,skill_range_17"].notna()]))
print("Has Support filled:", len(five_and_six_wiat[five_and_six_wiat["PreInt_EduHx,CPSE"].notna() | five_and_six_wiat["PreInt_EduHx,EI"].notna()]))
print("Has Prematurity filled:", len(five_and_six_wiat[five_and_six_wiat["PreInt_DevHx,premature"].notna() | five_and_six_wiat["PreInt_DevHx,premature"].notna()]))


print("\nHas CBCL filled:", len(five_and_six_wiat[five_and_six_wiat["CBCL,EID"].notna()]))
print("Has CBCL-Pre filled:", len(five_and_six_wiat[five_and_six_wiat["CBCL_Pre,EID"].notna()]))

Has Parent learning problems filled: 37
Has Spoke first word age filled: 562
Has Named objects age filled: 427
Has Put 2 or 3 words together age filled: 507
Has Support filled: 628
Has Prematurity filled: 552

Has CBCL filled: 487
Has CBCL-Pre filled: 147


In [28]:
# Correlation and multiple regression

for_corr = five_and_six_wiat[["PreInt_FamHx,f_learn",
                "PreInt_DevHx,skill_age_15",
                "PreInt_DevHx,skill_range_15",
                "PreInt_DevHx,skill_age_16",
                "PreInt_DevHx,skill_range_16",
                "PreInt_DevHx,skill_age_17",
                "PreInt_DevHx,skill_range_17",
                 "PreInt_EduHx,CPSE",
                 "PreInt_EduHx,EI",
                 "PreInt_DevHx,premature",
                 spelling,
                ]]

# Transform PreInt_DevHx,skill_range_X cols into a number (Early or early=-1, Normal or normal=0, Late or late=1, other=NaN)
cols = [x for x in for_corr.columns if "PreInt_DevHx,skill_range_" in x]   
dict = {"Early": -1, "early": -1, "Normal": 0, "normal": 0, "Late": 1, "late": 1}  
for_corr = LORIS_data_preparation.replace_with_dict_otherwise_nan(for_corr, cols, dict)

for col in for_corr.columns:
    for_corr[col] = for_corr[col].astype(float)

for col in for_corr.columns:
    df = for_corr[[col, spelling]].dropna()
    print(df.corr())
    
for_lr = for_corr.drop("PreInt_FamHx,f_learn", 1).dropna()
import statsmodels.api as sm
lr = sm.OLS(for_lr[spelling], for_lr.drop(spelling, 1)).fit()
print(lr.summary())

                      PreInt_FamHx,f_learn  WIAT,WIAT_Spell_Stnd
PreInt_FamHx,f_learn              1.000000             -0.061014
WIAT,WIAT_Spell_Stnd             -0.061014              1.000000
                           PreInt_DevHx,skill_age_15  WIAT,WIAT_Spell_Stnd
PreInt_DevHx,skill_age_15                   1.000000             -0.144039
WIAT,WIAT_Spell_Stnd                       -0.144039              1.000000
                             PreInt_DevHx,skill_range_15  WIAT,WIAT_Spell_Stnd
PreInt_DevHx,skill_range_15                     1.000000             -0.246855
WIAT,WIAT_Spell_Stnd                           -0.246855              1.000000
                           PreInt_DevHx,skill_age_16  WIAT,WIAT_Spell_Stnd
PreInt_DevHx,skill_age_16                   1.000000             -0.225388
WIAT,WIAT_Spell_Stnd                       -0.225388              1.000000
                             PreInt_DevHx,skill_range_16  WIAT,WIAT_Spell_Stnd
PreInt_DevHx,skill_range_16            

/Users/home/Repos/hbn/HBN-scripts/exploration/helpers/LORIS_data_preparation.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[~data[col].isin(dict.keys()), col] = np.nan # Replace any other possible values with NaN
/Users/home/Repos/hbn/HBN-scripts/exploration/helpers/LORIS_data_preparation.py:250: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].astype(str).replace(dict)
/var/folders/63/96f7chcx7qq5r9fq4x0x72bw0000gq/T/ipykernel_46811/2973943645.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [29]:
relevant_assessments_read = ["WISC", "PreInt_EduHx", "PreInt_DevHx", "PreInt_TxHx", "PreInt_Lang", "PreInt_FamHx", "PreInt_FamHx_RDC", 
                             "PreInt_Demos_Home", "PreInt_Demos_Fam", "NIH_Scores", "SympChck", "SCQ", "Barratt", 
        "ASSQ", "ARI_P", "SDQ", "SWAN", "SRS", "CBCL", "ICU_P", "ICU_SR", "PANAS", "APQ_P", "PCIAT", "DTS", "ESWAN", "MFQ_P", "APQ_SR", 
        "WHODAS_P", "CIS_P", "SAS", "PSI", "RBS", "PhenX_Neighborhood", "WHODAS_SR", "CIS_SR", "SCARED_P", "SCARED_SR", 
        "C3SR", "CCSC", "CPIC", "YSR", "PhenX_SchoolRisk", "CBCL_Pre", "SRS_Pre", "ASR"]
display(LORIS_data_preparation.get_cumul_df(five_and_six_wiat, relevant_assessments_read))

,Respondents,Assessments,Min Age,N of Assessments,Last Assessment
0,635,[PreInt_EduHx],5.0,1,PreInt_EduHx
1,634,"[PreInt_EduHx, PreInt_TxHx]",5.0,2,PreInt_TxHx
2,633,"[PreInt_EduHx, PreInt_TxHx, PreInt_DevHx]",5.0,3,PreInt_DevHx
3,630,"[PreInt_EduHx, PreInt_TxHx, PreInt_DevHx, Symp...",5.0,4,SympChck
4,627,"[PreInt_EduHx, PreInt_TxHx, PreInt_DevHx, Symp...",5.0,5,NIH_Scores
5,626,"[PreInt_EduHx, PreInt_TxHx, PreInt_DevHx, Symp...",5.0,6,SCQ
6,624,"[PreInt_EduHx, PreInt_TxHx, PreInt_DevHx, Symp...",5.0,7,SDQ
7,619,"[PreInt_EduHx, PreInt_TxHx, PreInt_DevHx, Symp...",5.0,8,Barratt
8,613,"[PreInt_EduHx, PreInt_TxHx, PreInt_DevHx, Symp...",5.0,9,PreInt_Demos_Fam
9,613,"[PreInt_EduHx, PreInt_TxHx, PreInt_DevHx, Symp...",5.0,10,PreInt_Demos_Home
